# EV Charging LLM Fine-tuning Pipeline
This notebook fine-tunes a small language model (≤7B parameters) on electric vehicle charging domain data using LoRA/QLoRA techniques.

In [ ]:
# Core imports
import json
import pandas as pd
import torch
import os
from datetime import datetime
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
import wandb
from sklearn.model_selection import train_test_split

In [ ]:
# Configuration
CONFIG = {
    "model_name": "microsoft/DialoGPT-small",  # Small model for demo
    "max_length": 512,
    "train_batch_size": 4,
    "eval_batch_size": 4,
    "learning_rate": 5e-4,
    "num_epochs": 3,
    "warmup_steps": 100,
    "logging_steps": 10,
    "save_steps": 500,
    "output_dir": "./fine_tuned_model",
    "data_path": "output_data/ev_training_alpaca.json",
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.1
}

print("Configuration loaded:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# Setup device and directories
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create output directories
os.makedirs(CONFIG["output_dir"], exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Initialize experiment tracking (optional)
experiment_name = f"ev_charging_llm_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
print(f"Experiment name: {experiment_name}")

## Data Loading and Preprocessing

In [ ]:
# Load training data
with open(CONFIG["data_path"], 'r', encoding='utf-8') as f:
    training_data = json.load(f)

print(f"Loaded {len(training_data)} training examples")

# Display first example
if training_data:
    print("\nFirst training example:")
    print(f"Instruction: {training_data[0]['instruction']}")
    print(f"Input: {training_data[0]['input']}")
    print(f"Output: {training_data[0]['output'][:200]}...")

In [ ]:
def format_prompt(example):
    """Format training examples into prompt-response format"""
    if example['input']:
        prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    
    response = example['output']
    return prompt + response + "</s>"

# Format all training examples
formatted_data = [format_prompt(example) for example in training_data]

print(f"Formatted {len(formatted_data)} examples")
print("\nFirst formatted example:")
print(formatted_data[0][:300] + "...")

In [ ]:
# Split data into train/validation
train_texts, val_texts = train_test_split(
    formatted_data, 
    test_size=0.1, 
    random_state=42
)

print(f"Training examples: {len(train_texts)}")
print(f"Validation examples: {len(val_texts)}")

## Model and Tokenizer Setup

In [ ]:
# Load tokenizer and model
print(f"Loading model: {CONFIG['model_name']}")

tokenizer = AutoTokenizer.from_pretrained(CONFIG["model_name"])
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name"],
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Model loaded with {model.num_parameters():,} parameters")
print(f"Tokenizer vocabulary size: {len(tokenizer)}")

In [ ]:
# Setup LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    target_modules=["c_attn", "c_proj"]  # For DialoGPT
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("\nLoRA configuration applied successfully")

## Data Tokenization

In [ ]:
def tokenize_function(examples):
    """Tokenize text examples"""
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=CONFIG["max_length"],
        return_tensors="pt"
    )

# Create datasets
train_dataset = Dataset.from_dict({"text": train_texts})
val_dataset = Dataset.from_dict({"text": val_texts})

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

print(f"Tokenized training dataset: {len(train_dataset)} examples")
print(f"Tokenized validation dataset: {len(val_dataset)} examples")

## Training Setup and Execution

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=CONFIG["output_dir"],
    num_train_epochs=CONFIG["num_epochs"],
    per_device_train_batch_size=CONFIG["train_batch_size"],
    per_device_eval_batch_size=CONFIG["eval_batch_size"],
    learning_rate=CONFIG["learning_rate"],
    warmup_steps=CONFIG["warmup_steps"],
    logging_steps=CONFIG["logging_steps"],
    save_steps=CONFIG["save_steps"],
    evaluation_strategy="steps",
    eval_steps=CONFIG["save_steps"],
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=None,  # Disable wandb for now
    logging_dir="./logs",
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=False
)

print("Training arguments configured")

In [ ]:
# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM, not masked LM
)

print("Data collator configured")

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

print("Trainer initialized successfully")
print(f"Training will run for {CONFIG['num_epochs']} epochs")
print(f"Total training steps: {len(train_dataset) // CONFIG['train_batch_size'] * CONFIG['num_epochs']}")

In [ ]:
# Start training
print("Starting fine-tuning...")
print(f"Start time: {datetime.now()}")

# Train the model
trainer.train()

print(f"Training completed at: {datetime.now()}")

## Model Saving and Export

In [ ]:
# Save the fine-tuned model
final_model_path = os.path.join(CONFIG["output_dir"], "final_model")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"Model saved to: {final_model_path}")

# Save training configuration
config_path = os.path.join(final_model_path, "training_config.json")
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

print(f"Training configuration saved to: {config_path}")

## Quick Model Testing

In [ ]:
# Test the fine-tuned model
def generate_response(prompt, max_length=200):
    """Generate response using the fine-tuned model"""
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):].strip()

# Test with EV charging questions
test_prompts = [
    "### Instruction:\nWhat are the benefits of fast charging for electric vehicles?\n\n### Response:\n",
    "### Instruction:\nHow do I find charging stations near me?\n\n### Response:\n",
    "### Instruction:\nWhat is the difference between AC and DC charging?\n\n### Response:\n"
]

print("Testing fine-tuned model:")
print("=" * 50)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\nTest {i}:")
    print(f"Prompt: {prompt.split('### Response:')[0].split('### Instruction:')[1].strip()}")
    response = generate_response(prompt)
    print(f"Response: {response}")
    print("-" * 30)

## Training Summary

In [ ]:
# Training summary
print("=== Fine-tuning Summary ===")
print(f"Base model: {CONFIG['model_name']}")
print(f"Training examples: {len(train_texts)}")
print(f"Validation examples: {len(val_texts)}")
print(f"Training epochs: {CONFIG['num_epochs']}")
print(f"LoRA rank: {CONFIG['lora_r']}")
print(f"LoRA alpha: {CONFIG['lora_alpha']}")
print(f"Learning rate: {CONFIG['learning_rate']}")
print(f"Model saved to: {final_model_path}")
print(f"Experiment: {experiment_name}")

# Get final training metrics
if hasattr(trainer.state, 'log_history') and trainer.state.log_history:
    final_metrics = trainer.state.log_history[-1]
    print("\nFinal metrics:")
    for key, value in final_metrics.items():
        if isinstance(value, (int, float)):
            print(f"  {key}: {value:.4f}")

print("\n✅ Fine-tuning completed successfully!")